In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Raw Data Visualizations and Null Value Dropping

Since this is a huge dataset, we'll only be able to work with a fraction of the data. The whole dataset is 55 million rows, we'll work with 3 million rows.

We'll first check for any oddities in the data, before we transform it into something we can work with.

In [ ]:
train = pd.read_csv('../input/train.csv', nrows = 3000000)
test = pd.read_csv('../input/test.csv')
train.head(13)

Immediately we notice something odd: there's a row with all the spatial coordinates equal to 0. Let's see how many times this occurs in our sample of 5.5M rows, and if there's any other information we can get about those rows:

In [ ]:
train[(train['pickup_longitude'] == 0) | (train['pickup_latitude'] == 0) | (train['dropoff_longitude'] == 0) | (train['dropoff_latitude'] == 0)].head()
# We'll check if any coordinate is equal to 0. Nowhere in the continental United States does lat or long = 0, so having 0 lat or long anywhere is pretty unreasonable.

In [ ]:
print('Ratio of null rows: {}'.format(len(train[(train['pickup_longitude'] == 0) | (train['pickup_latitude'] == 0) | (train['dropoff_longitude'] == 0) | (train['dropoff_latitude'] == 0)]) / len(train)))

In [ ]:
ax = train[(train['pickup_longitude'] == 0) | (train['pickup_latitude'] == 0) | (train['dropoff_longitude'] == 0) | (train['dropoff_latitude'] == 0)]['fare_amount'].plot.hist(bins = 50)
ax.set_title('Fare Amounts for Null Rows')

So based on that analysis, here's what we know:
* About 2% of the rows contain null spatial data
* Those rows still contain valid target values (fare_amount), which are mostly small values but include some large/very large amounts, as well as some negative amounts.

Given this information, I think it would make the most sense to drop these rows from the dataset. Although 2% of the data is nontrivial, the lack of spatial information makes these data points difficult to work with. The spatial data is going to be key in predicting the target, and I would not want to skew the correct data by imputing these null rows with incorrect data. Therefore, we will drop these rows.

Let's visualize some other features in the raw dataset:

In [ ]:
ax = train['fare_amount'].plot.hist(bins = 50)
ax.set_title('Fare Amounts Distribution')

In [ ]:
train['fare_amount'].describe()

Looks as expected, fares mostly below $20.

Some oddities include:
* There are negative fares. It's not really clear how to interpret a negative fare, so we will most likely drop those rows from the dataset.
* The maximum fare is $1200. Based on the histogram that seems like an outlier, but we will have to inspect the dataset further to see what's going on with that point.

In [ ]:
train['passenger_count'].value_counts().sort_index().plot.bar(title = 'Distribution of Passenger Counts')

We see a non-trivial number of trips with 0 passengers, and some trips with 51, 129, or 208 passengers.

In [ ]:
train[train['passenger_count'] == 0].head()

Aside from the passenger count, nothing seems too off about these rows. The fares and distances traveled are pretty normal-looking. 

One theory I've seen is that customers will call a taxi, which starts the meter running, then cancel before they get picked up, thus racking up a fare without ever being a passenger. We can check later if the distribution of distances/fares/other metrics are different for 0 passenger trips versus non-zero passenger trips. This will help us decide if we want to leave 0 passenger counts as a level in this category, or perhaps impute it with 1.

Trips with 0, 51, 129, or 208 passengers are certainly odd, but if passenger count doesn't have an effect on the fare then we don't need to worry about this feature. We'll do more inspection later.

Let's see the spatial distribution of the datapoints:

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (14,5))
train_nonzero = train.drop(list(np.where((train['pickup_longitude'] == 0) | 
                          (train['pickup_latitude'] == 0) | 
                          (train['dropoff_longitude'] == 0) | 
                          (train['dropoff_latitude'] == 0))[0]), axis = 0)
train_nonzero['dropoff_longitude'].describe()
ax1.scatter(x = train_nonzero['pickup_longitude'], y = train_nonzero['pickup_latitude'])
ax2.scatter(x = train_nonzero['dropoff_longitude'], y = train_nonzero['dropoff_latitude'])
ax1.set_title('Pickup Coordinates')
ax2.set_title('Dropoff Coordinates')

This doesn't seem right. Looking at the data, we can tell that NYC coordinates are centered around latitude 41 longitude -74, so it's odd to see points ranging from -3000:3000.

One thing to note is that we need a metric for deciding what coordinates are outliers. We can't just look at a histogram plot to decide what an outlier is. So we need to decide now how we're going to indentify coordinate outliers.

The way we'll do that is using the test set. If we look at the distributions of the coordinates in the test set, they're totally normal:

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (12,6))
ax1.scatter(x = test['pickup_longitude'], y = test['pickup_latitude'])
ax2.scatter(x = test['dropoff_longitude'], y = test['dropoff_latitude'])
ax1.set_title('Pickup Coordinates')
ax2.set_title('Dropoff Coordinates')

So, we'll use the max/min coordinates of the test set as our guidelines for what isn't an outlier. To make sure we're not restricting ourselves too much, we'll add a buffer of 1.4 lat/long units on every end (although this isn't a huge buffer, the vast majority of datapoints occur within this region). This amounts to a buffer of about 100 miles on each end, which is pretty generous, considering we expect the vast majority of our data to be centered in New York City. The following code will trim the fat:

In [ ]:
buffer = 1.4
x_max = max(test['pickup_longitude'].max(),test['dropoff_longitude'].max()) + buffer
x_min = min(test['pickup_longitude'].min(),test['dropoff_longitude'].min()) - buffer
y_max = max(test['pickup_latitude'].max(),test['dropoff_latitude'].max()) + buffer
y_min = min(test['pickup_latitude'].min(),test['dropoff_latitude'].min()) - buffer

train_non0 = train.drop(list(np.where((train['pickup_longitude'] == 0) | 
                          (train['pickup_latitude'] == 0) | 
                          (train['dropoff_longitude'] == 0) | 
                          (train['dropoff_latitude'] == 0))[0]), axis = 0)
train_non0.reset_index(inplace = True)
train_non0.drop('index', axis=1, inplace=True)

outliers_index = list(np.where(
    (train_non0['pickup_longitude'] > x_max) | (train_non0['dropoff_longitude'] > x_max) |
    (train_non0['pickup_longitude'] < x_min) | (train_non0['dropoff_longitude'] < x_min) |
    (train_non0['pickup_latitude'] > y_max) | (train_non0['dropoff_latitude'] > y_max) |
    (train_non0['pickup_latitude'] < y_min) | (train_non0['dropoff_latitude'] < y_min)
)[0])
train_outliers = train_non0.drop(index = outliers_index)

fig, (ax1,ax2) = plt.subplots(1,2, figsize = (14,5))
ax1.scatter(x = train_outliers['pickup_longitude'], y = train_outliers['pickup_latitude'])
ax2.scatter(x = train_outliers['dropoff_longitude'], y = train_outliers['dropoff_latitude'])
ax1.set_title('Pickup Coordinates')
ax2.set_title('Dropoff Coordinates')

Now the coordinates look about right. We sacrificed some data, but it's more important that we have a large amount of useable data.

## Data Processing and Preliminary Feature Extraction

That's about all we can visualize without doing further processing on the data.
Since this is a huge dataset, we're going to create a processing function that can create some features from the dataset piecewise, so that we don't have the read in all the data at once. Since the preprocessing is meant to be done in chunks, there are no aggregation features done here, only features that could be extracted from a single row.
Some features that this function extracts:
* Drop rows with empty (0) coordinates
* Drop rows with NaN's
* Drop rows with coordinates too far outside of NYC
* Drop rows with negative fare
* Converts string date to datetime
* Extract the year, month, day, dayofweek, and hour as features
* Extract the vector between pickup and dropoff points
* Get the manhattan distance and euclidean distance between pickup and dropoff (may do haversine also)
* Get the angle of the direction vector (relative to east). For ML, this will be encoded as sine and cosine of said angle. Also extracts the compass direction of the vector, in case it's useful.
* Calculate the fare per distance of the trip, which can be used as a crude pricing metric.

In [ ]:
## Preprocessing function
## Apply this function to the dataset/subset of the dataset
## Transformations that can be done elementwise (and don't require aggregations of the whole dataset) occur here
def data_transform1(df):
    ## Reset the index
    df.reset_index(inplace = True)
    df.drop('index', axis = 1, inplace = True)
    ## Drop NA's
    df.dropna(axis = 0, how = 'any', inplace = True)
    df.reset_index(inplace = True)
    df.drop('index', axis = 1, inplace = True)
    ## Turns out some rows contain 0 for the latitude/longitude data. Since these rows are of no use to us, we will drop them
    df.drop(index = list(np.where((df['pickup_longitude'] == 0) | 
                          (df['pickup_latitude'] == 0) | 
                          (df['dropoff_longitude'] == 0) | 
                          (df['dropoff_latitude'] == 0))[0]), inplace = True)
    df.reset_index(inplace = True)
    df.drop('index', axis = 1, inplace = True)
    ## Define conditions for outliers. test must already be defined in memory. buffer is flexible.
    buffer = 1.4
    x_max = max(test['pickup_longitude'].max(),test['dropoff_longitude'].max()) + buffer
    x_min = min(test['pickup_longitude'].min(),test['dropoff_longitude'].min()) - buffer
    y_max = max(test['pickup_latitude'].max(),test['dropoff_latitude'].max()) + buffer
    y_min = min(test['pickup_latitude'].min(),test['dropoff_latitude'].min()) - buffer
    ## Drop outliers
    df.drop(index = list(np.where(
        (df['pickup_longitude'] > x_max) | (df['dropoff_longitude'] > x_max) |
        (df['pickup_longitude'] < x_min) | (df['dropoff_longitude'] < x_min) |
        (df['pickup_latitude'] > y_max) | (df['dropoff_latitude'] > y_max) |
        (df['pickup_latitude'] < y_min) | (df['dropoff_latitude'] < y_min)
        )[0]), inplace = True)
    df.reset_index(inplace = True)
    df.drop('index', axis = 1, inplace = True)
    ## Drop rows with negative fare
    df.drop(list(np.where(df['fare_amount'] < 0)[0]), inplace = True)
    df.reset_index(inplace = True)
    df.drop('index', axis = 1, inplace = True)
    ## Convert to datetime
    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], format = '%Y-%m-%d %H:%M:%S %Z')
    ## Create new features based on datetime
    df['year'] = df['pickup_datetime'].map(lambda x:x.year)
    df['month'] = df['pickup_datetime'].map(lambda x:x.month)
    df['day'] = df['pickup_datetime'].map(lambda x:x.day)
    df['dayofweek'] = df['pickup_datetime'].map(lambda x:x.dayofweek)
    df['hour'] = df['pickup_datetime'].map(lambda x:x.hour)
    ## Create distance vectors that we will use to create other features - these features will be dropped probably
    df['x_diff'] = df['dropoff_longitude'] - df['pickup_longitude']
    df['y_diff'] = df['dropoff_latitude'] - df['pickup_latitude']
    ## Manhattan Distance
    df['manhattan_distance'] = df['y_diff'].abs() + df['x_diff'].abs()
    ## Euclidean Distance
    df['distance'] = np.sqrt(df['y_diff'] ** 2 + df['x_diff'] ** 2)
    ## Fare per distance
    df['fare_per_distance'] = df['fare_amount'].divide(df['distance'])
    ## Direction. After some research, it appears the best way to encode this feature for machine learning is with two features:
    ## The sine and cosine of the angle (relative to east).
    df['cosine'] = df['x_diff'].divide(df['distance'])
    df['sine'] = df['y_diff'].divide(df['distance'])
    ## We will also create an angle feature, just in case it's useful later.
    df['angle'] = (np.arccos(df['cosine']) * (180 / np.pi) * df['y_diff'].map(lambda x: 1 if x == 0 else np.sign(x)) + 360) % 360
    ## We can also convert an angle from degrees into a compass direction.
    df['compass_direction'] = df['angle'].map(degree_to_compass)
    

def degree_to_compass(angle):
    if np.isnan(angle):
        return(np.nan)
    dirs = ['E','ENE','NE','NNE','N','NNW','NW','WNW','W','WSW','SW','SSW','S','SSE','SE','ESE']
    ix = int((angle + 11.25) / 22.5 - 0.02)
    return dirs[ix % 16]

In [ ]:
data_transform1(train)
train.head()

One thing I discovered while writing that function is there are some rows where the distance traveled is 0.

In [ ]:
train[train['distance'] == 0].head()

In these cases the angle gets reported as NaN, since there is no angle. In how many rows does this occur:

In [ ]:
print('Out of {} rows, {} have distance of 0'.format(len(train), len(train[train['distance'] == 0])))

So about 1% of rows have this phenomenon. Let's see if there's anything peculiar about those rows:

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (10,3))
ax1.hist(train[train['distance'] == 0]['fare_amount'], bins = 50)
ax2.hist(train[train['distance'] != 0]['fare_amount'], bins = 50)
ax1.set_title('Fares with distance 0')
ax2.set_title('Fares with distance >0')

We can't see much of a difference in the rows with distance 0. It might make sense to drop these rows like we dropped the null rows (especially since there are so few of them). However, these rows do actually contain spatial data and fares, so they may be useful. We'll keep these rows for now, and we'll decide what to do with them after a bit more analysis.

Let's check some other univariate distribution statistics to try to find anything interesting:

In [ ]:
fig, (ax1,ax2,ax3,ax4,ax5) = plt.subplots(1,5, figsize = (16,5))
ax1.bar(train['year'].unique(), train.groupby('year').mean()['fare_amount'].values)
ax2.plot(train[(train['distance'] > 0.05) & (train['distance'] < 0.6)].groupby(['year','month']).mean()['fare_per_distance'].values)
ax3.plot(train[(train['distance'] > 0.05) & (train['distance'] < 0.6)].groupby('hour').mean()['fare_per_distance'].values)
ax4.plot(train[(train['distance'] > 0.05) & (train['distance'] < 0.6)].groupby('dayofweek').mean()['fare_per_distance'].values)
ax5.hist(train['distance']*69, bins = 50)
ax1.set(title = 'Mean fare per year', xlabel = 'Year')
ax2.set(title = 'Mean rate per month, 2009-2015')
ax3.set(title = 'Mean rate by hour', xlabel = 'Hour of day')
ax4.set(title = 'Mean rate by day of week', xlabel = 'Day of Week')
ax5.set(title = 'Distribution of trip distance', xlabel = 'Distance (mi., approx)', xlim = (0,20))

Here, rate is the fare divided by the distance travelled in a trip. This value may be skewed due to some long-distance/low-fare trips (which is why in the code I restricted distances to under 40 miles for this graph), but we will later see that most of the data follows a linear trend.

What we can see from these graphs:
* The mean trip price has been increasing each year, more or less. Based on the first graph, we will probably want to use the year as a feature in our model.
* The mean trip rate per month has an interesting jump right in the middle - maybe the taxi company raised their prices that month. There doesn't appear to be a strong seasonality so we won't use a time series for our model. But maybe we should add a variable for 'After September 2012', since that increase in fare rate is non-trivial.
* There is a clear seasonality in the fare per distance metric by hour of the day. Fares get a lot more expensive around 9AM until 6PM. We will probably want to use a sine transform of the hour of the day to capture this feature.
* There appears to be a dependence of the rate on the day of the week. But if you look at the y-axis of that graph, the range in values is so small compared to the other 2 that the effect is not worth trying to model, as we may overfit by introducing this feature.

There are some outliers in distance that we can see here:

In [ ]:
train['distance'].sort_values().tail() * 69

Some trips that hit the boundary of our defined region. These are pretty long for taxi rides, but not totally unreasonable.

Now let's try visualizing some interactions between variables.
The most fundamental interaction we would expect to see is a positive correlation between ride distance and fare amount. We'll start with a scatter plot of those variables:

In [ ]:
fig,ax = plt.subplots(figsize = (14,6))
g = sns.regplot(x = 'distance', y = 'fare_amount', data = train, ax = ax)
g.set_title('Fares vs. Distance')

The bulk of the data follows a linear trend.

The longer trips (>0.8 distance, or about 50mi.+) all have constant fare. It's likely that these trips are all fixed-fare trips: for example, a trip to the airport may be constant regardless of the distance. We will probably want to introduce a 'long_trip' flag for trips longer than 0.8, as these trips are most likely not going to follow the linear trend (there are a small number of exceptions that you can see, but out of 2 million points it's a very small number). However, the length of the trip may not be the only metric that indicates a constant fare. Pickup and dropoff locations are probably also important.

In addition, those distance 0 trips are throwing off the data a bit. The fares range from 0 to $500, and without knowing the distance traveled they aren't contributing to this particular model.

There's also some short trips with fares of over $400. I'm not sure how that happens, so we might drop those points as outliers.

What would our regression plot look like if we dropped the distance 0 trips, and the longer constant-fare trips:

In [ ]:
fig,ax = plt.subplots(figsize = (14,6))
g = sns.regplot(x = 'distance', y = 'fare_amount', data = train[(train['distance'] > 0.02) & (train['distance'] < 0.55) & (train['fare_amount'] > 1) & (train['fare_amount'] < 220)], ax = ax)
g.set_title('Fares vs. Distance (No outliers)')

It's messy, but there is a linear trend for datapoints within this range.

We can see some low-fare trips that range pretty large distances (30 miles for under $10 for example), which seem like they will be difficult to predict if we are only given the start and end coordinates.

For completeness, let's see if there is a similar or better trend when using the manhattan distance:

In [ ]:
fig,ax = plt.subplots(figsize = (14,6))
g = sns.regplot(x = 'manhattan_distance', y = 'fare_amount', data = train[(train['manhattan_distance'] > 0.02) & (train['manhattan_distance'] < 0.65) & (train['fare_amount'] > 1) & (train['fare_amount'] < 220)], ax = ax)
g.set_title('Fares vs. Manh. Distance (No outliers)')

Looks about the same. It's not clear if euclidean or manhattan distance (or both, or a weighted average, etc) will produce a better model. We will have to wait till the modeling stage.

Let's see how some metrics vary by the number of passengers. Recall the distribution of passenger counts:

In [ ]:
train['passenger_count'].value_counts()

There's some pretty clear outliers here. We will first check the non-outlier values:

In [ ]:
f, (ax1,ax2) = plt.subplots(1,2, figsize = (14,5))
for p in list(range(0,7)):
    sns.kdeplot(train[train['passenger_count'] == p]['distance']*69, ax = ax1, label = "Passengers: {}".format(p))
    sns.kdeplot(train[train['passenger_count'] == p]['fare_amount'], ax = ax2, label = "Passengers: {}".format(p))
ax1.set(xlabel = 'Distance (mi., approx)', title = 'KDE of Distance by # Passengers', xlim = (0,6))
ax2.set(xlabel = 'Fare Amount (USD)', title = 'KDE of Fare Amount by # Passengers', xlim = (0,25))

In [ ]:
train[train['distance'] > 0.05].groupby('passenger_count').mean()['fare_per_distance']

Based on these, we can see:
* With the exception of 0-passenger trips, distance travelled per trip seems inversely related to the number of passengers (1 and 2 passengers travel longer distances)
* In correspondance with the previous bullet, 1 and 2 passenger trips tend to pay more per trip (because they travel further)
* However, the mean fare per distance travelled is roughly the same for all passenger counts.

So, it appears that the number of passenger doesn't actually affect the fare. The only correlation is between distance travelled and total fare, and the rate of this correlation doesn't depend on the number of passengers. Therefore, we can probably ignore the number of passengers when we build a model.

Next we'll try and visualize the actual locations of pickup and dropoff points using the coordinates. We will do this using a 2d hexplot.
The objective is to try to find distinct "hotspots." If we can find some groups of locations with similar fare values or trends, then we can create bins to sort the pickup and dropoff locations into, and hopefully those bins act as better features than the coordinates themselves.

In [ ]:
sns.jointplot(train['pickup_longitude'], train['pickup_latitude'], kind = 'kde', bins = 1000, xlim = (-74.1,-73.8), ylim = (40.6,41.0))
sns.jointplot(train['dropoff_longitude'], train['dropoff_latitude'], kind = 'kde', bins = 1000, xlim = (-74.1,-73.8), ylim = (40.6,41.0))

These plots alone don't quite tell us what we need to know about the location data. A high degree of sophistication is required to work with the coordinates; I'll come back to it when I have a better idea of how to glean knowledge from them.

## Examining the Test Set

We're almost ready to create our first version of a model. However, after completing our analyses we still have some points we need to address when modeling, such as:
* How do we handle the number of passengers (esp. outliers like 0 passengers or >6 passengers)
* How do we handle distances of 0
* How do we handle distances >30 miles (at which point the linear relation between fare and distance ceases to exist)
* Should we model with euclidean distance or manhattan distance (or both)?

To answer these questions, we'll start by looking at the test set. We want to see the distribution of data points in the test set, so we can try to model our interpretation of the training set around it.

The first thing we need to do is apply our transformation function to the test set to get the distances and other features.

In [ ]:
## Have to give a 'fare_amount' column to the test set so the function will not error.
test['fare_amount'] = 0
data_transform1(test)
test['fare_amount'] = np.nan
test.head()

Distribution of passenger count:

In [ ]:
test['passenger_count'].value_counts().sort_index().plot.bar(title = 'Distribution of Passenger Counts')

No 0-passenger rides present in the test set. But we probably aren't going to use passenger count in our model anyways.

Distribution of distances of trips:

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2,figsize = (12,4))
ax1.hist(test['distance']*69, bins = 30)
ax2.hist(test['manhattan_distance']*69, bins = 40)
ax1.set(title = 'Trip Distance Distribution', xlabel = 'Distance (mi. approx)')
ax2.set(title = 'Trip Manhattan Distance Distribution', xlabel = 'Distance (mi. approx)')

The minimum distance in the test set is:

In [ ]:
print('Min distance: {}'.format(test['distance'].min()))
print('Min manhattan distance: {}'.format(test['manhattan_distance'].min()))

So the test set does indeed contain some distance 0 trips. How may?

In [ ]:
print('Out of {} rows, {} have distance of 0'.format(len(test), len(test[test['distance'] == 0])))

Like the training set, we have to deal with about 1% of rows having distance 0. Therefore, we won't drop those distance 0 trips from the training set, since we'll need them to make predictions. Instead, we'll create a 'distance0' variable that will indicate if the trip has distance 0, and see if the model can use that to make an accurate prediction.

In [ ]:
test['distance'].sort_values(ascending = False)

From the test set histograms we see that there are a very small number of points with distance >30 mi. There are in fact only three trips with distances greater than 30 mi., and they are all about 70 mi. Looking at the training data, most of the test points (which have distance <30 mi.) lie in the linear region of the fare vs. distance plot. The 70 mile points lie in a region that is almost entirely constant-fare. We can assume these 3 points are probably constant-fare as well, and we can create a 'long_distance' boolean variable just for these 3 points.

## Model V2.2

Model V2 introduces new Distance == 0 and Distance > 0.8 boolean variables. Model V2 will also not drop distance 0 points from the training set.

Model V2.1 introduces a bias column.

Model V2.2 introduces the 'After Sept 2012' variable, to reflect the jump in fare price in September 2012.

In [ ]:
test.head()

We will try XGBoost for our model. The features we will use to predict are:
* Pickup coordinates
* Dropoff coordinates
* Year
* Sine transform of the hour of the day
* Distance
* Manhattan Distance
* Distance == 0 (new from V1)
* Distance > 0.8 (new from V1)
* Cosine and Sine of direction traveled

In [ ]:
import xgboost as xgb
from sklearn.preprocessing import StandardScaler

In [ ]:
def data_transform2(df):
    ## Bias column
    df['bias'] = 1
    ## Hour of the day variable
    df['sine_hour'] = np.sin(2 * np.pi * df['hour'] / 24)
    ## Distance 0 variable
    df['distance0'] = (df['distance'] == 0)
    ## Long distance variable
    df['long_distance'] = (df['distance'] > 0.8)
    ## After September 2012
    df['after_sept2012'] = (df['year'] > 2012) | ((df['year'] == 2012) & (df['month'] >= 9))

In [ ]:
## Drop the training rows where distance = 0
target = train['fare_amount']

cols = ['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','sine_hour','distance','manhattan_distance','cosine','sine']

## Concatenate train and test for scaling
## (Can de-concatenate by checking for NaN rows in fare_amount)
df = pd.concat([train,test], ignore_index = True, sort = False)
train_index = np.where(~(pd.isnull(df['fare_amount'])))
test_index = np.where(pd.isnull(df['fare_amount']))

## New sine-hour feature
data_transform2(df)

## Scale
scaler = StandardScaler()
df = pd.concat([pd.DataFrame(data = scaler.fit_transform(df[cols]), columns = cols),df[['year','distance0','long_distance','bias','after_sept2012']]], axis = 1)
train = df.loc[train_index]
test = df.loc[test_index]

Data preprocessing is done. Let's create a model.

In [ ]:
model = xgb.XGBRegressor(max_depth = 7, learning_rate = 0.012, n_estimators = 300)
model.fit(X = train, y = target, eval_metric = 'rmse')
pred = model.predict(data = test)

In [ ]:
submission = pd.read_csv('../input/sample_submission.csv')
submission['fare_amount'] = pred
submission.to_csv('submissionV2_2.csv', index = False)

This crude version ended up with a ~4.5 RMSE, about 1 unit better than the basic linear model. It's good that we're better than the baseline, but we should aim to do better.

Some things we can do to improve the model include:
* Measure the popularity of a pickup/dropoff point: fraction of points that are within some distance (distance will need to be tuned) of the pickup point/dropoff point (debatable if this has an effect on fare though)
* Find the mean fare of points close to the pickup/dropoff point (again have to find if this actually impacts the fare) (this feature may encode tolls though so it could be worthwhile)
* Distance of pickup/dropoff to airport (or, boolean for within some miles of an airport) This may encode the constant fare trips.
* Use more data. We used about 4% of the data. Either construct models piecewise and take an average, or try to read in the whole dataset somehow.
